# Finance ROIC

In [7]:
import os
import dotenv

dotenv.load_dotenv()
token = os.getenv('TUSHARE_TOKEN')

In [8]:
import tushare as ts
import pandas as pd

In [9]:
ts.set_token(token)
pro = ts.pro_api()

In [10]:
# 获取当前所有A股上市公司列表
stock_list = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [11]:
stock_list

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,国华网安,深圳,软件服务,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427
...,...,...,...,...,...,...
5336,873703.BJ,873703,广厦环能,北京,专用机械,20231205
5337,873726.BJ,873726,卓兆点胶,江苏,机械基件,20231019
5338,873806.BJ,873806,N云星宇,北京,软件服务,20240111
5339,873833.BJ,873833,美心翼申,重庆,汽车配件,20231108
